## 20. Data Engineering

#### Data Engineering Overview

**What is Data Engineering?**  

Data science is a team sport. There are many different team roles, including: 

* Data Scientists
* Business Architects
* Data Architects
* Data Visualizers
* Data Engineers

Data scientists are only as good as the data they have access to. Most companies store their data in variety of formats across databases and text files. This is where data engineers come in — they build pipelines that transform that data into formats that data scientists can use. Data engineers are just as important as data scientists, but tend to be less visible because they tend to be further from the end product of the analysis and deal with a lot of the less glamourous underpinning elements as show in the diagram below.

![Data Science Hierarchy of Needs](./images/ds-hierarchy-of-needs.png)

This framework puts things into perspective. Before a company can optimize the business more efficiently or build data products more intelligently, layers of foundational work need to be built first.  

The field of Data Engineering could be thought of as a superset of business intelligence and data warehousing that brings more elements from software engineering. This discipline also integrates specialization around the operation of so called “big data” distributed systems, along with concepts around the extended Hadoop ecosystem, stream processing, and in computation at scale.

Among the many valuable things that Data Engineers do, one of their highly sought-after skills is the ability to design, build, and maintain data warehouses. Just like a retail warehouse is where consumable goods are packaged and sold, a data warehouse is a place where raw data is transformed and stored in query-able forms.

**What does a Data Engineer do?**

As Data Science exploded, Data Engineering was emerging as a complimentary discipline, taking cues from its sibling, while also defining itself in opposition, and finding its own identity. Like Data Science, Data Engineering is also a broad field, but any individual data engineer doesn't need to know the whole spectrum of skills, instead being 'T-shaped' with a single deep specialisation and a broad overall knowledge. A Data Engineer needs to be good at:  

* Architecting distributed systems
* Creating reliable pipelines
* Combining data sources
* Architecting data stores
* Collaborating with data science teams and building the right solutions for them

**A brief note on Infrastructure**

In smaller teams or organisations, where no data infrastructure team has yet been formalized, the data engineering role may also cover the workload around setting up and operating the organization’s data infrastructure. This includes tasks like setting up and operating platforms like Hadoop/Hive/HBase, Spark, and the like. This would usually be designed by a data architect and built by a data engineer (Architects design, Engineers build).


#### Data Modeling

**Data Modeling**

Most user-centric systems are optimised for lots of simple, small transactions. These are usually **OnLine Transaction Processing (OLTP)** systems. However when querying large amounts of data, systems need to be optimised for large, complex transactions. These are called **OnLine Analytical Processing (OLAP)** systems. This analytics-first approach often involves a design process called [Data Modeling](Data Modelling) and refers to the practice of documenting software and business system design. Again, in larger organisations this would be the job of a data architect but sometimes falls to Data Engineers also. The “modeling” of these various systems and processes often involves the use of diagrams, symbols, and textual references to represent the way the data flows through a software application or the Data Architecture within an enterprise. This is expressed through an [Entity Relationship Diagram](https://www.lucidchart.com/pages/ER-diagram-symbols-and-meaning) (ERD) as follows:
![Data Model](./images/data-modelling.gif)

*PK = Primary Key FK = Foreign Key*

The first thing that needs to be done is to determine the extent to which data should be [Normalised](https://en.wikipedia.org/wiki/Database_normalization). This involves organising the columns (attributes or variables) and tables (relations). Normalised tables have simpler schemeas, more standardised data and carry less redundency. However this comes at the trade off of more code, joins and pipelines. Additionally wider tables are easier to query because they are pre-joined, however this is less efficient and slower and makes pipelines more difficult to create and run.

**STAR Schema**

One of the most commonly used patterns to balance Normalisation is the [STAR Schema](https://en.wikipedia.org/wiki/Star_schema), so called because of the star-like pattern in which they are organised. This design focuses on building normalized tables, specifically fact and dimension tables. When needed, denormalized tables can be built from these smaller normalized tables. This design strives for a balance between ETL maintainability and ease of analytics.

![STAR Schema](./images/star.png)

**Fact & Dimension Tables**

To understand how to build denormalized tables from fact tables and dimension tables, we need to discuss their respective roles in more detail:

* Fact tables typically contain point-in-time transactional data. Each row in the table can be extremely simple and is often represented as a unit of transaction. Because of their simplicity, they are often the source of truth tables from which business metrics are derived. For example, at Airbnb, we have various fact tables that track transaction-like events such as bookings, reservations, alterations, cancellations, and more.

* Dimension tables typically contain slowly changing attributes of specific entities, and attributes sometimes can be organized in a hierarchical structure. These attributes are often called “dimensions”, and can be joined with the fact tables, as long as there is a foreign key available in the fact table. At Airbnb, we built various dimension tables such as users, listings, and markets that help us to slice and dice our data.

**Data Partitioning & Backfilling Historical Data**

We live in an era where data storage costs are low and processing power is cheap and that allows companies to store their historical data rather than throwing it away. This means that companies can re-process their data in response to new changes.

[Data Partitioning](https://www.blendo.co/blog/sql-table-and-data-partitioning-how-to/) is where the data is broken up into self-contained chunk with a partition key, which allows the data to be looked very quickly and can greatly improve performance. One common partition key to use is the **datestamp** (or ds for short). Raw data is often organised and ordered by date and batch ETL jobs are usually run each day also which means new date partitions are created for each daily run. Finally, many queries will often involve a time period so the date is a natural choice to partition our data by.

**Backfilling**

Another important advantage of using datestamp as the partition key is the ease of data backfilling. When a ETL pipeline is built, it computes metrics and dimensions forward, not backward. Often, we might desire to revisit the historical trends and movements. In such cases, we would need to compute metric and dimensions in the past — We called this process data backfilling. Backfilling is so common that Hive built in the functionality of dynamic partitions, a construct that perform the same SQL operations over many partitions and perform multiple insertions at once.

#### ETL & Pipelines

**ETL**

[ETL](https://www.quora.com/What-is-ETL) stands for Extract, Transform and Load, the three stages in gathering and preparing data for storing for analysis.  **Extract** is the stage at which data is extracted from other homogeneous or heterogeneous data, **Transform** is where the data is transformed for storing in the proper format or structure for the purposes of querying and analysis, and **Load** is where the data is loaded into the target database. ETL used to be conducted through drap and drop interfaces, however more recently, there has been a shift towards a more programatic approach as the prevailing wind is that code is the best abstraction there is for software, although this is a contentious argument. Real life ETL jobs can be complex ranging from deriving business events from [CRUD Operations](https://en.wikipedia.org/wiki/Create,_read,_update_and_delete), or computing features for a machine learning model on a daily basis to predict something of business value.

![ETL_Pipeline](./images/etl.png)

This processes can also be called [Data Pipelining](https://medium.com/the-data-experience/building-a-data-pipeline-from-scratch-32b712cfb1db) and the two terms are often interchangable or combined, however the term pipelining will often refer to the larger process of which ETL is a part.

![Data Pipeline](./images/pipeline.png)

**ETL Frameworks**

Until recently much of the work in ETL was scheduled using [CRON jobs](https://support.hostgator.com/articles/cpanel/what-are-cron-jobs) in Linux, however over the last few years, new and open source frameworks for pipelining and ETL have emerged including [Azkaban](https://azkaban.github.io/azkaban/docs/latest/) from LinkedIn, [Luigi](https://github.com/spotify/luigi) from Spotify and [Airflow](https://airflow.apache.org/) from Airbnb.

Regardless of which framework you choose there are some important factors to consider:

* Configuration: ETLs are naturally complex, and we need to be able to succinctly describe the data flow of a data pipeline. As a result, it is important to evaluate how ETLs are authored. Is it configured on a UI, a domain specific language, or code? Nowadays, the concept of configuration as code is gaining popularity, because it allows users to expressively build pipelines programmatically that are customizable.  

* UI, Monitoring, Alerts: Long running batch processes inevitably can run into errors (e.g. cluster failures) even when the job itself does not have bugs. As a result, monitoring and alerting are crucial in tracking the progress of long running processes. How well does a framework provide visual information for job progress? Does it surface alerts or warnings in a timely and accurate manner?

* Backfilling: Once a data pipeline built, we often need to go back in time and re-process the historical data. Ideally, we do not want to build two separate jobs, one for backfilling historical data and another for computing current or future metrics. How easy does a framework support backfilling? Can it do so in a way that is standardized, efficient, and scalable? All these are important questions to consider.

#### Building a Pipeline in Airflow

**Installing Airflow**

You can install Airflow as follows:

`pip install apache-airflow`

**Defining the Directed Acyclic Graph (DAG)**

As mentioned previously, ETL is made up of three parts: **Extract**, **Transform** & **Load**. This can be quite a complicated process and is best understoof through a graph. Visually, a node in a graph represents a task, and an arrow represents the dependency of one task on another. Given that data only needs to be computed once on a given task and the computation then carries forward, the graph is directed and acyclic. This is why Airflow jobs are commonly referred to as “DAGs” (Directed Acyclic Graphs).

![DAG](./images/dag.png)

One of the clever designs about Airflow UI is that it allows any users to visualize the DAG in a graph view, using code as configuration. The author of a data pipeline must define the structure of dependencies among tasks in order to visualize them. This specification is often written in a file called the DAG definition file, which lays out the anatomy of an Airflow job.

**Operators: Sensors, Operators, and Transfers**

While DAGs describe how to run a data pipeline, operators describe what to do in a data pipeline. Typically, there are three broad categories of operators:

* Sensors: waits for a certain time, external file, or upstream data source
* Operators: triggers a certain action (e.g. run a bash command, execute a python function, or execute a Hive query, etc)
* Transfers: moves data from one location to another

**Sensors** unblock the data flow after a certain time has passed or when data from an upstream data source becomes available.  
**Operators** trigger data transformations, which corresponds to the Transform step. Because Airflow is open-source, contributors can extend BaseOperator class to create custom operators as they see fit. At Airbnb, the most common operator we used is HiveOperator (to execute hive queries), but we also use PythonOperator (e.g. to run a Python script) and BashOperator (e.g. to run a bash script, or even a fancy Spark job) fairly often. The possibilities are endless here!  
**Transfers** move data from one place to another, which often maps to the Load step in ETL. At Airbnb, we use MySqlToHiveTransfer or S3ToHiveTransfer pretty often, but this largely depends on one’s data infrastructure and where the data warehouse lives.


**ETL Best Practice**

Partition Data Tables: As we mentioned earlier, data partitioning can be especially useful when dealing with large-size tables with a long history. When data is partitioned using datestamps, we can leverage dynamic partitions to parallelize backfilling.

Load Data Incrementally: This principle makes your ETL more modular and manageable, especially when building dimension tables from the fact tables. In each run, we only need to append the new transactions to the dimension table from previous date partition instead of scanning the entire fact history.

Enforce Idempotency: Many data scientists rely on point-in-time snapshots to perform historical analysis. This means the underlying source table should not be mutable as time progresses, otherwise we would get a different answer. Pipeline should be built so that the same query, when run against the same business logic and time range, returns the same result. This property has a fancy name called Idempotency.

Parameterize Workflow: Just like how templates greatly simplified the organization of HTML pages, Jinja can be used in conjunction with SQL. As we mentioned earlier, one common usage of Jinja template is to incorporate the backfilling logic into a typical Hive query. Stitch Fix has a very nice post that summarized how they use this technique for their ETL.

Add Data Checks Early and Often: When processing data, it is useful to write data into a staging table, check the data quality, and only then exchange the staging table with the final production table. At Airbnb, we call this the stage-check-exchange paradigm. Checks in this 3-step paradigm are important defensive mechanisms — they can be simple checks such as counting if the total number of records is greater than 0 or something as complex as an anomaly detection system that checks for unseen categories or outliers.

Build Useful Alerts & Monitoring System: Since ETL jobs can often take a long time to run, it’s useful to add alerts and monitoring to them so we do not have to keep an eye on the progress of the DAG constantly. Different companies monitor DAGs in many creative ways — at Airbnb, we regularly use EmailOperators to send alert emails for jobs missing SLAs. Other teams have used alerts to flag experiment imbalances. Yet another interesting example is from Zymergen where they report model performance metrics such as R-squared with a SlackOperator.

#### Big & Emerging Data

**Hadoop**

[Hadoop](http://hadoop.apache.org/) is a framework that allows for the distributed processing of large data sets across clusters of computers using simple programming models. It is designed to scale up from single to clusters of thousands of machines (also called **nodes**), each offering local computation and storage. Rather than rely on hardware to deliver high-availability, the library itself is designed to detect and handle failures at the application layer, so delivering a highly-available service on top of a **cluster** of nodes, each of which may be prone to failures.

**HDFS**  
[HDFS](https://hortonworks.com/apache/hdfs/) stands for Hadoop Distributed File System. This is a method Hadoop uses for storing large amounts of data (Terabytes or Petabytes) across a large number of individual nodes. 

**Mapreduce**
[Mapreduce](https://hortonworks.com/apache/mapreduce/) is the original framework for writing applications that process large amounts of structured and unstructured data stored in the Hadoop Distributed File System (HDFS). Apache Hadoop YARN opened Hadoop to other data processing engines that can now run alongside existing MapReduce jobs to process data in many different ways at the same time.

**Spark**

Spark is a general-purpose data processing engine that is suitable for use in a wide range of circumstances. Application developers and data scientists incorporate Spark into their applications to rapidly query, analyze, and transform data at scale. Tasks most frequently associated with Spark include interactive queries across large data sets, processing of streaming data from sensors or financial systems, and machine learning tasks.

Despite common opinion, Spark cannot be compared directly to Hadoop, but [should instead be compared to HDFS](https://www.xplenty.com/blog/apache-spark-vs-hadoop-mapreduce/). Both HDFS and Spark have been designed for slightly different purposes with Hadoop being better at operations that require writing outputs to disk and Spark being better at in-memory operations where all the data being processed will fit into memory. 

**Graph Databases**

We live in a connected world! There are no isolated pieces of information, but rich, connected domains all around us. Only a database that natively embraces relationships is able to store, process, and query connections efficiently. While other databases compute relationships at query time through expensive JOIN operations, a graph database stores connections as first class citizens.

Accessing nodes and relationships in a native graph database is an efficient, constant-time operation and allows you to quickly traverse millions of connections per second per core.

Independent of the total size of your dataset, graph databases excel at managing highly connected data and complex queries. Armed only with a pattern and a set of starting points, graph databases explore the larger neighborhood around the initial starting points — collecting and aggregating information from millions of nodes and relationships — leaving the billions outside the search perimeter untouched.

Below is a visual example:

![Graph Database](./images/graph.svg)

**Sources & Further Reading**

[Beginners Guide to Data Engineering Part 1](https://medium.com/@rchang/a-beginners-guide-to-data-engineering-part-i-4227c5c457d7)<br/>
[Beginners Guide to Data Engineering Part 2](https://medium.com/@rchang/a-beginners-guide-to-data-engineering-part-ii-47c4e7cbda71)<br/>
[Beginners Guide to Data Engineering Part 3](https://medium.com/@rchang/a-beginners-guide-to-data-engineering-the-series-finale-2cc92ff14b0)<br/>
[What is Data Engineering?](https://www.dataquest.io/blog/what-is-a-data-engineer/)<br/>
[Data Engineer DDAT Description](https://www.gov.uk/government/publications/data-engineer-role-description/data-engineer-role-description--2)<br/>
[The Rise of the Data Engineer](https://medium.freecodecamp.org/the-rise-of-the-data-engineer-91be18f1e603)<br/>
[Data Pipelining](https://medium.com/the-data-experience/building-a-data-pipeline-from-scratch-32b712cfb1db)<br/>
[Apache Airflow](http://michal.karzynski.pl/blog/2017/03/19/developing-workflows-with-apache-airflow/)<br/>
[Best Practices with Airflow](https://gtoonstra.github.io/etl-with-airflow/principles.html)<br/>